In [9]:
from data_utils import train_loader, test_loader
import torch, transformers
from diffusers import StableDiffusionXLPipeline

In [ ]:
# --- 1. Environment Info ---

print("torch:", torch.__version__)
print("diffusers:", diffusers.__version__)
print("transformers:", transformers.__version__)


# Load SDXL pipeline with fp16 weights
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    variant="fp16",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

# --- 3. Inspect Components ---
print("Pipeline components:", list(pipe.components.keys()))

print("\\nUNet:\n", pipe.unet)
print("\\nVAE:\n", pipe.vae)
print("\\nText Encoder 1:\n", pipe.text_encoder)
print("\\nText Encoder 2:\n", pipe.text_encoder_2)
print("\\nTokenizer:\n", pipe.tokenizer)

# --- 4. Count Parameters ---
total_params = sum(p.numel() for p in pipe.unet.parameters())
trainable_params = sum(p.numel() for p in pipe.unet.parameters() if p.requires_grad)
print(f"UNet total params: {total_params:,}")
print(f"UNet trainable params: {trainable_params:,}")


torch: 2.7.0+cu126
diffusers: 0.33.1
transformers: 4.52.3


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Pipeline components: ['vae', 'text_encoder', 'text_encoder_2', 'tokenizer', 'tokenizer_2', 'unet', 'scheduler', 'image_encoder', 'feature_extractor']
\nUNet:
 UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (add_time_proj): Timesteps()
  (add_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=2816, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 320, eps=1e-05, affine=True)
          (conv1): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Minimal safe LoRA injection: inject into a single known attention block

from diffusers.models.attention_processor import LoRAAttnProcessor

# Target 1 specific known block: down_blocks[1].attentions[0].transformer_blocks[0].attn2
attn_module = pipe.unet.down_blocks[1].attentions[0].transformer_blocks[0].attn2

# Extract dimensions
hidden_size = attn_module.to_q.in_features
cross_attention_dim = attn_module.to_k.in_features

# Create LoRA processor and apply
lora_processor = LoRAAttnProcessor(
    hidden_size=hidden_size,
    cross_attention_dim=cross_attention_dim,
    rank=4,
    scale=1.0,
)

attn_module.set_processor(lora_processor)

# Check trainable parameters
trainable_params = sum(p.numel() for p in attn_module.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in attn_module.parameters())
trainable_pct = round(100 * trainable_params / total_params, 4)

{
    "Module path": "down_blocks[1].attentions[0].transformer_blocks[0].attn2",
    "Hidden size": hidden_size,
    "Cross-attn dim": cross_attention_dim,
    "Total params": total_params,
    "Trainable (LoRA) params": trainable_params,
    "Trainable %": trainable_pct
}



Found 140 attention processors to check...
Processing: down_blocks.1.attentions.0.transformer_blocks.0.attn1.processor
❌ Failed to inject LoRA at down_blocks.1.attentions.0.transformer_blocks.0.attn1.processor: LoRAAttnProcessor.__init__() got an unexpected keyword argument 'hidden_size'
Processing: down_blocks.1.attentions.0.transformer_blocks.0.attn2.processor
❌ Failed to inject LoRA at down_blocks.1.attentions.0.transformer_blocks.0.attn2.processor: LoRAAttnProcessor.__init__() got an unexpected keyword argument 'hidden_size'
Processing: down_blocks.1.attentions.0.transformer_blocks.1.attn1.processor
❌ Failed to inject LoRA at down_blocks.1.attentions.0.transformer_blocks.1.attn1.processor: LoRAAttnProcessor.__init__() got an unexpected keyword argument 'hidden_size'
Processing: down_blocks.1.attentions.0.transformer_blocks.1.attn2.processor
❌ Failed to inject LoRA at down_blocks.1.attentions.0.transformer_blocks.1.attn2.processor: LoRAAttnProcessor.__init__() got an unexpected key

ValueError: Mismatch in processor count – some injections failed.